In [1]:
from langchain_unstructured import UnstructuredLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re
from rank_bm25 import BM25Okapi
from langchain.schema import BaseRetriever  # or from langchain.retrievers.base import BaseRetriever
from langchain.schema import BaseRetriever
from typing import Any, List, Tuple
from langchain.docstore.document import Document
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
import os
from dotenv import load_dotenv
from langchain.llms import OpenAI, HuggingFaceHub
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

import getpass
os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
os.environ["LANGSMITH_TRACING"] = "true"

/Users/elisimicrobertson/Documents/relevance_asssessment/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO: pikepdf C++ to Python logger bridge initialized


In [3]:
import os
from dotenv import load_dotenv
from langchain.embeddings import OpenAIEmbeddings, CohereEmbeddings

load_dotenv()  # Load the environment variables from .env

def get_embedding_model(model_name: str):
    """
    Factory function to return an embedding model instance.
    
    Supported model_name values:
      - "openai": Returns an OpenAIEmbeddings instance.
      - "cohere": Returns a CohereEmbeddings instance.
    """
    if model_name.lower() == "openai":
        openai_api_key = os.getenv("OPENAI_API_KEY")
        if not openai_api_key:
            raise ValueError("OPENAI_API_KEY is not set in the environment.")
        return OpenAIEmbeddings(openai_api_key=openai_api_key)
    elif model_name.lower() == "cohere":
        cohere_api_key = os.getenv("COHERE_API_KEY")
        if not cohere_api_key:
            raise ValueError("COHERE_API_KEY is not set in the environment.")
        return CohereEmbeddings(cohere_api_key=cohere_api_key)
    else:
        raise ValueError(f"Unsupported embedding model: {model_name}")

In [5]:
embedding_model = get_embedding_model("openai")

# Build the dense vector store using FAISS.
# FAISS provides similarity_search_with_score which we'll use in our hybrid retriever.
vector_store = FAISS.from_documents(elements, embedding_model)

dense_retriever = vector_store

# Initialize BM25 retriever
bm25_retriever = BM25Retriever(elements)

# Create a HybridRetriever instance with a chosen weight.
# For example, alpha=0.5 gives equal weight to dense and BM25 scores.
hybrid_retriever = HybridRetriever(bm25_retriever=bm25_retriever, 
                                   dense_retriever=dense_retriever,
                                   alpha=0.5)

llm = ChatOpenAI(temperature=0, openai_api_key=os.getenv("OPENAI_API_KEY"), model="gpt-4o")

query = "What are the key coverages in the insurance policy?"
print("Query:", query)

qa_pipeline = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=hybrid_retriever)

/var/folders/d_/d28rwb_10zd4841zb_1t_fmw0000gn/T/ipykernel_88448/1973275175.py:19: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  return OpenAIEmbeddings(openai_api_key=openai_api_key)
INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO: Loading faiss.
INFO: Successfully loaded faiss.
INFO: Failed to load GPU Faiss: name 'GpuIndexIVFFlat' is not defined. Will not load constructor refs for GPU indexes.


Query: What are the key coverages in the insurance policy?


/var/folders/d_/d28rwb_10zd4841zb_1t_fmw0000gn/T/ipykernel_88448/758316775.py:18: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0, openai_api_key=os.getenv("OPENAI_API_KEY"), model="gpt-4o")


In [ ]:


judge_llm = init_chat_model("gpt-4o-mini", openai_api_key=os.getenv("OPENAI_API_KEY"), model_provider="openai")
judge_llm = judge_llm.with_structured_output(JudgeResponse)
results = llm_judge_evaluation(dataset, qa_pipeline, judge_llm)

In [ ]:
# Initialize the chain
qa_chain = LongQAChain(
    retriever=hybrid_retriever,
    llm=ChatOpenAI(model_name="gpt-3.5-turbo")
)

# Ask a question
answer = qa_chain.run("Summarise the policies of this document?")

/var/folders/d_/d28rwb_10zd4841zb_1t_fmw0000gn/T/ipykernel_88448/2639158125.py:121: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  documents = self.retriever.get_relevant_documents(question)
INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 